In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set()
import os
import random 

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# read in original data
train_data = "hida_workspace/trainSet/trainSet.txt"
test_data = "hida_workspace/testSet/testSet.txt"

df_train = pd.read_csv(train_data)
df_train_length_idx = len(df_train)
df_test = pd.read_csv(test_data)

# Merge the two datasets
train_test = [df_train, df_test]
df_train_test = pd.concat(train_test)
# df_train_test['Prognosis'].loc[df_train_test['Prognosis'] == '<undefined>'] = np.nan
df_train_test.loc[df_train_test['Prognosis'] == '<undefined>', 'Prognosis'] = np.nan

# all variables in the dataset (incl. outcome)
variables = list(df_train_test.columns[3:])


# read in csv from Tobias (imputed via catboost
df_imputed_train = pd.read_csv("Results/Tobias/imputed_train.csv")
df_imputed_test = pd.read_csv("Results/Tobias/imputed_test.csv")


df_for_rf = df_train.copy()
df_for_rf["Prognosis"] = df_for_rf["Prognosis"].astype("category").cat.codes
df_for_rf = df_for_rf[variables]











# get predictions from xgboost imputed:
xgboost_features = variables.copy()
xgboost_features.remove("Prognosis")

train_valid_split = 0.9
sep_index = int(863 * train_valid_split)

X_train = df_imputed_train[xgboost_features].iloc[0:sep_index]
y_train = df_imputed_train["Prognosis"].iloc[0:sep_index]

X_test = df_imputed_train[xgboost_features].iloc[sep_index:]
y_test = df_imputed_train["Prognosis"].iloc[sep_index:]


# fit the model 
#model = CatBoostClassifier(verbose=0, iterations=1000, task_type="GPU", devices='0:1')
#model.fit(X_train.values, y_train.values)


# fit model no training data
model = XGBClassifier()
model.fit(X_train.values, y_train.values)

# make a prediction
#yhat = model.predict(X_test.values)

#catboost_imputed_results = dict(predictions_cb_imputed=list(yhat), images=image_names.to_list(), targets=y_test.values)
#catboost_imputed_results_df = pd.DataFrame(catboost_imputed_results)

[11:37:38] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/haicore-project-covchl/do8557/.local/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=20, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [3]:
print(model)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=20, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)


In [4]:
# make predictions for test data
y_pred = model.predict(X_test.values)
xgboost_imputed_results = dict(predictions_xg_imputed=list(y_pred), images=image_names.to_list(), targets=y_test.values)
xgboost_imputed_results_df = pd.DataFrame(xgboost_imputed_results)

/home/haicore-project-covchl/do8557/.local/lib/python3.6/site-packages/xgboost/data.py:115: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "memory consumption")


NameError: name 'image_names' is not defined

776    1.0
777    1.0
778    0.0
779    1.0
780    1.0
      ... 
858    1.0
859    1.0
860    0.0
861    1.0
862    0.0
Name: Prognosis, Length: 87, dtype: float64